# RedGrease Demo at RedisConf 2021

Quick demonstration of how to create and run Redis Gears functions, using RedGrease.

## Demos:
1. [The Basics](#demo-basics)
2. [Simple Analytics Query](#demo-query)
3. [Transactions Stream Processing](#demo-stream)
4. [Custom Command](#demo-command)


# Preparations
Before running the demos, make sure that the prerequisites are met and that the preparation steps have successfully been executed. 
Some preparation steps, particularly the downloads, may take quite some time. 

## 1. Prerequisites
- Python3.7
- Pip
- Docker
- Jupyter

Run the cell below tho validate your prerequisites.

In [1]:
import sys
import re
pyver = !{sys.executable} --version  # type: ignore
pipver = !{sys.executable} -m pip --version  # type: ignore
dockver = !docker --version  # type: ignore

if not re.match("Python 3.7", pyver[0]):
    raise SystemExit("This demo only supports Python 3.7. ")

if not re.match(".*\(python 3.7\)", pipver[0]):
    raise SystemExit("Please install Pip for yout Python 3.7 environment. ")

if not re.match("Docker version", dockver[0]):
    raise SystemExit("Please install Docker")

print("Requirements all look good!")

Requirements all look good!


## 2. Python Requirements

Install the Python packages required for the demo:

- `redgrease[client]` - The RedGrease client library for Redis Gears. This is what is being demonstrated.

- `ipywidgets` - Jupyter notebook exetension, for displaying widgets, e.g. buttons, in this notebook.
- `requests` - For downloading content.

Run the cell below to install the requirements.

In [2]:
!{sys.executable} -m pip install redgrease[client] ipywidgets requests  # type: ignore
!jupyter nbextension enable --py widgetsnbextension  # type: ignore

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


## 3. Download Datasets
Some of the demos requiere a portion of the [COCO Dataset](https://cocodataset.org) to be uploaded into the Redis Gears Cluster.
The COCO Dataset (Common Objects in Context) is a fairly large set of (~247,000) images and corresponding annotations of what tey are depicting.

### Example:
<img src="coco_example.jpg" > [COCO Example](coco_example.jpg)

```
a man riding a snowboard down a ski slope.
a snowboarder sailing down a snowy hillside on a mountain.
a man is snowboarding past blue markers on a mountain.
a man on a snowboard in the snow.
a man snow boarding in the snow on a slope. 
```


For the demo we will only pre-download the annotations (json), not the images (jpeg), but it is still between 250 - 500 MB of data, depending on which portions you choose.

There are two annotation packages to choose from. 
- **COCO Train/Cal 2014** - Annotations for 124,000 images (241 MB)
- **COCO Train/Val 2017** - Annotations for 123,000 images (241 MB)

Either or both may be used. 
Run the cell below and select using the buttons which dataset(s) to download.

In [3]:
# This code is just for preparation of the demo.
# It is NOT part of the demo itself
#
# Download COCO Annotations 
# Run the cell, then:
# - Validate or modify the Download directory
# - Click the button, or buttons for the annotations to download

import ipywidgets as widgets
import os
import requests

coco_annotations_url = "http://images.cocodataset.org/annotations"
annotations_file_pattern = "annotations_trainval{}.zip"

layout = widgets.Layout(width="30%")
output = widgets.Output()

def get_download_path():
    download_dir = "."
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            download_dir = winreg.QueryValueEx(key, downloads_guid)[0]
    else:
        download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

    return os.path.join(download_dir, "COCO")

download_location = widgets.Text(
    value=get_download_path(),
    placeholder="Download directory",
    description="Directory to download annotations to.",
    layout=layout,
)
display(download_location)

def dl_state(button, downloading=None):
    year = button.value
    annotations_file_name = annotations_file_pattern.format(year)
    destination = os.path.join(download_location.value, annotations_file_name)
    is_downloaded = os.path.isfile(destination)
    button.disabled = is_downloaded or downloading is not None
    if downloading:
        button.description=f"Downloading COCO {year} annotations (241 MB): {downloading}%. Please wait!"
    elif is_downloaded:
        button.description=f"Congrats! COCO {year} annotataions is downloaded!"
    else:
        button.description=f"Download COCO {year} annotations (241 MB)"
    return is_downloaded, annotations_file_name, destination


def download_button_pressed(btn):
    downloaded, file_name, destination = dl_state(btn)
    if downloaded:
        return
    if not os.path.isdir(download_location.value):
        os.mkdir(download_location.value)
    try:
        response = requests.get(
            f"{coco_annotations_url}/{file_name}",
            stream=True
        )
        total_length = response.headers.get('content-length')
        with open(destination, "wb") as f:
            if total_length is None: # no content length header
                dl_state(btn, "???")
                f.write(response.content)
                return
            total_length = int(total_length)
            dl = 0
            for data in response.iter_content(chunk_size=4096):
                dl += len(data)
                f.write(data)
                dl_state(btn, int(100*(dl/total_length)))

    except Exception:
        try:
            os.remove(destination)
        except Exception:
            pass
    finally:
        dl_state(btn)

for year in ["2014", "2017"]:
    download_button = widgets.Button(
        tooltip='Start download of selected datasets into the selected download directory.',
        layout=layout
    )
    download_button.value = year
    dl_state(download_button)
    download_button.on_click(download_button_pressed)
    display(download_button)

display(output)

Text(value='/home/anders/Downloads/COCO', description='Directory to download annotations to.', layout=Layout(w…

Button(description='Congrats! COCO 2014 annotataions is downloaded!', disabled=True, layout=Layout(width='30%'…

Button(description='Congrats! COCO 2017 annotataions is downloaded!', disabled=True, layout=Layout(width='30%'…

Output()

## 4. Download and run Redis Gears Cluster Docked image
Run the cell below to download a Redis Gears Cluster Docker image (~605 MB), if not already present, and run it. 

In [6]:
redis_gears_cluster_image = "redislabs/rgcluster:1.0.6"
redis_gears_cluster_container_name = "demo_gears_cluster"

redis_gears_single_image = "redislabs/redisgears:1.0.6"
redis_gears_single_container_name = "demo_gears_single"

# Get the correct Redis Gears Images
!docker pull {redis_gears_single_image}
!docker pull {redis_gears_cluster_image}

# Check if the single container is already running.
container_info = !docker container inspect {redis_gears_single_container_name}
if container_info[0] == "[]":
    print("Starting Redis Gears single instance")
    !docker run --name {redis_gears_single_container_name} --rm -d -p 6379 {redis_gears_single_image}


# Check if the cluster container is already running.
container_info = !docker container inspect {redis_gears_cluster_container_name}
if container_info[0] == "[]":
    print("Starting Redis Gears cluster instance")
    !docker run --name {redis_gears_cluster_container_name} --rm -d -p 30001:30001 -p 30002:30002 -p 30003:30003 {redis_gears_cluster_image}

print("Redis Gears containers are running!")

1.0.6: Pulling from redislabs/redisgears
Digest: sha256:ab126c449864cc9bc1b1facae26069c51d5f56760fb15b7a01371317fc69c17f
Status: Image is up to date for redislabs/redisgears:1.0.6
docker.io/redislabs/redisgears:1.0.6
1.0.6: Pulling from redislabs/rgcluster
Digest: sha256:563b2bf890085dc0b451de536fb7bacf8bfb98b6d1a162fda92f3b4742c4fd23
Status: Image is up to date for redislabs/rgcluster:1.0.6
docker.io/redislabs/rgcluster:1.0.6
Starting Redis Gears single instance
e0e86d92238579860102e2971c65a2adc378113e63a9e3416113d74bc29b7cfb
Starting Redis Gears cluster instance
3bcbae03bab0eaac887e4a215ac3cac1afc8989d90478bdb02966c0e4bb30d5b
Redis Gears containers are running!


## 5. Load Annotation Data into Redis cluster
By running the cell below, the COCO annotations downloaded above will be loaded into the Redis Cluster.

In [31]:
import glob
import itertools
import json
import os
import re
import redgrease
import zipfile

annotation_archive_files = os.path.join(download_location.value, "annotations_trainval*.zip")
annotation_archives = glob.glob(annotation_archive_files)

if not annotation_archives:
    print("no archives")
    raise SystemExit("Please download either or both COCO annotations as per instructions above.")

r = redgrease.RedisGears(host="localhost", port=30001)

annotation_json_pattern = re.compile("annotations/(\w+)_(\w+).json")

annotation_types = ["instances"] #  , "person_keypoints", "captions"]
datasets = ["val2014"]  # , "train2014", "val2017", "train2017"]

output = widgets.Output()

def load_annotation_info(info, dataset_name, annotation_type):
    annotation_info_key = f"/dataset/COCO/annotations/{annotation_type}/{dataset_name}/info"
    r.hset(annotation_info_key, mapping=info)
    return annotation_info_key

def load_licence_info(licence):
    licence_key = f"/licence/{licence['id']}"
    if not r.exists(licence_key):
        r.hset(licence_key, mapping=licence)
    return licence_key

def load_image_info(image_info):
    img_info_key = f"/dataset/COCO/image/{image_info['id']}/info"
    if not r.exists(img_info_key):
        r.hset(img_info_key, mapping=image_info)
    return img_info_key

def load_keypoint_names(base_key, keypoints):
    keypoints_key = f"{base_key}/keypoints"
    r.lpush(keypoints_key, *keypoints)
    return keypoints_key

def load_list_of_str(base_key, sequence):
    list_key = f"{base_key}/skeleton"
    r.lpush(list_key, *map(str, sequence))
    return list_key

def load_category(category):
    category_key = f"/dataset/COCO/annotations/category/{category['id']}"

    if "keypoints" in category:
        category["keypoints"] = load_keypoint_names(category_key, category["keypoints"])
    if "skeleton" in category:
        category["skeleton"] = load_list_of_str(category_key, category["skeleton"])

    r.hset(category_key, mapping=category)
    return category_key

def load_segmentation(annotation_key, segmentation):
    segmentation_key = f"{annotation_key}/segmentation"
    if not r.exists(segmentation_key):
        for i, segment in enumerate(segmentation):
            segment_key = f"{segmentation_key}/{i}"
            r.lpush(segment_key, *segment)
            r.rpush(segmentation_key, segment_key)
    return segmentation_key

def load_annotation(annotation, dataset_name, annotation_type):
    annotation_key = f"/dataset/COCO/annotations/{annotation_type}/{dataset_name}/annotation/{annotation['id']}"
    
    if not r.exists(annotation_key):
        if "segmentation" in annotation:
            # Replace the 'segmentation' list-of-lists, with a key with a list of keys, that in turn point to the inner lists :)
            annotation["segmentation"] = load_segmentation(annotation_key, annotation["segmentation"])
        
        if "bbox" in annotation:
            # Replace the 'bbox' with a string reepresentaton.laad_segmentation
            annotation["bbox"] = str(annotation["bbox"])

        if "keypoints" in annotation:
            annotation["keypoints"] = load_list_of_str(annotation_key, annotation["keypoints"])

        r.hset(annotation_key, mapping=annotation)
    return annotation_key

def load_annotation_jsons_from_zip(zip_file):
    with zipfile.ZipFile(zip_file) as archive:
        for file_name in archive.namelist():
            is_annotation_file = annotation_json_pattern.match(file_name)
            if not is_annotation_file:
                continue
            
            annotation_type = is_annotation_file.group(1)
            dataset_name = is_annotation_file.group(2)

            if not annotation_type in annotation_types:
                continue
            
            if not dataset_name in datasets:
                continue

            with archive.open(file_name) as json_file:
                contents = json.load(json_file)

            # info
            if "info" in contents:
                load_annotation_info(contents["info"], dataset_name, annotation_type)

            # licenses
            if "licenses" in contents:
                for lic in contents["licenses"]:
                    load_licence_info(lic)
            
            # images
            if "images" in contents:
                for image_info in contents["images"]:
                    load_image_info(image_info)

            # annotations
            if "annotations" in contents:
                for annotation in contents["annotations"]:
                    load_annotation(annotation, dataset_name, annotation_type)

            # categories (for "instances" and "person_keypoints")
            if "categories" in contents:
                for category in contents["categories"]:
                    load_category(category)

            
display(output)
for archive in annotation_archives:
    load_annotation_jsons_from_zip(archive)


Output()

# Demos
This is the actual Demo section. Everything above is just preparations.

<a id="deom-basics"></a>
## 1. The Basics

In [35]:
print("Demo 1 - The Basics")

Demo 1 - The Basics


<a id="demo-query"></a>
## 2. Simple Analytics Query 

In [34]:
print("Demo 2 - Analytical Query")

Demo 2 - Analytical Query


<a id="demo-stream"></a>
## 3. Transaction Stream Processing

In [33]:
print("Demo 3 - Transaction Stream Processing")

Demo 3 - Transaction Stream Processing


<a id="demo-command"></a>
## 4. Custom Command

In [32]:
print("Demo 4 - Custom Command")

Demo 4 - Custom Command
